In [1]:
import matplotlib.pyplot as plt 
%matplotlib inline 
import numpy as np 
import math
from scipy import linalg as la
import copy as cp 

This cell is all of the set up funcitons used to reproduce the analytical solution.

In [2]:
def pickmax(a):
    
    #initialize the maximum value and its indices, big=-1 to gaurentee the > is met at least once
    big = -1. 
    indi = 0
    indj = 0
    
    #iterate over matrix (input a) to find largest value, i+1 in j to get i != j w/o extra if statement

    for i in range(len(a)):
        for j in range(i+1,len(a)):
            if abs(a[i][j])> big:
                big = abs(a[i][j])
                indi = i
                indj = j
    return big, indi, indj

def settrig(a,p,q):
    
    #initialize tan, sin,l and cos respectively  
    t = s = c = 0
    
    #handles the non-singular case
    if a[p][q] != 0:
        tau = (a[q][q]-a[p][p])/(2*a[p][q])
    
        if tau < 0:
             t = -(-tau+math.sqrt(1+tau**2))**(-1.)
        else: 
            t = (tau+math.sqrt(1+tau**2))**(-1.)
        c = (math.sqrt(1+t**2))**(-1.)
        s = t*c

    else:
        c = 1.0
        s = 0.0
    return t,c,s

def jacobi(a):
    
    #find inputs to jacobi algo 
    maxval, p, q = pickmax(a)
    t,c,s = settrig(a,p,q)
    
    #note effective 'call by value' 
    b = cp.copy(a)
    
    #jacobi iteration 
    for i in range(len(a)):
        if i!=p or i!=q:
            #rotation 
            b[i][p] = a[i][p]*c-a[i][q]*s
            b[i][q] = a[i][q]*c+a[i][p]*s
            #force symmetry
            b[p][i] = b[i][p]
            b[q][i] = b[i][q]
    #force realtions for diagonal vals & maxval
    b[p][p] = a[p][p]*c**2-2*a[p][q]*c*s+a[q][q]*s**2
    b[q][q] = a[p][p]*s**2+2*a[p][q]*c*s+a[q][q]*c**2
    b[p][q] = b[q][p] = 0
    
    return b,maxval

def esolver(matrix, tolerance):
    
    #initailize & gaurentee 1  iteration 
    maxval = tolerance+1
    
    #iterate until tolerable 
    while tolerance<=maxval:
        matrix, maxval =jacobi(matrix )
        
    return matrix


def makemat(pmax,omega,n):
    p = np.linspace(0,pmax, num = n)

    hh = (n/(p[-1] - p[0]))**2
    hh2 = 2*hh
    #this cuts off the endpoints 
    A = np.zeros(shape = (n-1,n-1))
    V = np.zeros(n+1)


    for i in range(n):
        V[i] = omega**2*p[i]**2+1/p[i]

    for i in range(n-1):
            A[i][i] = hh2+V[i+1]
    #we evaluate v[i+1] so that we avoid the 0 point 
    #dont overwrite (n-2)
    for i in range(n-2):
            A[i+1][i]= A[i][i+1] = -hh
    return A


#output of the solver can be sloppy, lets clean it up.  
def clean(a):
    evals  = []
    #pick the diagonals
    for i in range(len(a)):
        evals.append(a[i][i])
        
    #sorted for easy reading 
    #and /2 to match expected values 
    sorted_evals  = np.sort(evals)/2
    return sorted_evals

The below cells show the construction and manipulation of the matrix to get the final answer

In [3]:
print(clean(esolver(makemat(40,.25,40),.00001)))

[  0.62181583   1.06526775   1.47963271   1.85087367   2.14990337
   2.40874635   2.77068912   3.22976528   3.76667366   4.37568805
   5.05435585   5.80137175   6.61596051   7.49762833   8.44604449
   9.4609788   10.54226605  11.68978461  12.90344296  14.18317098
  15.52891409  16.94062916  18.41828171  19.96184385  21.57129279
  23.24660973  24.98777904  26.79478762  28.66762441  30.60628002
  32.61074643  34.68101672  36.81708493  39.0189459   41.28659509
  43.6200286   46.01925596  48.48604367  51.10945762]


/home/mroosa/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:87: RuntimeWarning: divide by zero encountered in double_scalars


- This matches expectation 
> que celebration
- For further comments and tests of each of the functions see the build notebook 